# LangChain Semantic Search / Knowledge-Base Demo 🦜️🔗  

This notebook shows how to build a **searchable knowledge base** and perform **retrieval-augmented generation (RAG)** using LangChain + OpenAI embeddings and local vector store.  

## Takeaways
The whole LangChain pipeline is modular!! LangChain’s design made it easy to swap:  
- different vector stores (in-memory, pinecone, ..)  
- different embedding models (openai, azure, gemini, huggingface, ..) 
- different document loaders,  
- different LLMs for RAG.  
This flexibility is one of the biggest strengths of the framework.

### Compliance options for generating embeddings
- OpenAI API (default): Secure, encrypted, but hosted on OpenAI’s servers. Good for general use.
- Azure OpenAI Service: Same models, but hosted in Microsoft Azure with enterprise-grade compliance (GDPR, HIPAA, FedRAMP, etc.).
- Local embeddings (Hugging Face, etc.): Full control, no data leaves your environment, but requires more compute resources.

Based on the LangChain “Build a semantic search engine” tutorial:  
https://docs.langchain.com/oss/python/langchain/knowledge-base#openai  
## What this notebook does  
- Load documents (e.g. PDF, text, markdown) using LangChain’s document-loader abstractions.  
- Optionally split large documents into smaller chunks for better retrieval granularity.  
- Embed the text chunks into numeric vectors via OpenAI embeddings.  
- Store embeddings + metadata in a vector store for efficient similarity search.  
- Query the vector store: given a user query, retrieve relevant document snippets.
- Creates a RAG agent

In [ ]:
# !pip install langchain-community pypdf langchain-core langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [langchain]/7 [langgraph]


In [2]:
# Load env variables for LANGSMITH tracing and OPENAI_API_KEY
from dotenv import load_dotenv
import os 

load_dotenv()

from langsmith import traceable

### Document Loading

In [3]:
from langchain_community.document_loaders import PyPDFLoader

filepath = "./datasets/nke-10k-2023.pdf"
loader = PyPDFLoader(filepath)

# Langchain PyPDFLoader loads each page as a separate doc
docs = loader.load()
print(len(docs))

107


In [4]:
print(f"{docs[10].page_content[:100]}\n")
print(docs[10].metadata)

Table of Contents
INFORMATION ABOUT OUR EXECUTIVE OFFICERS
The executive officers of NIKE, Inc. as o

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './datasets/nke-10k-2023.pdf', 'total_pages': 107, 'page': 10, 'page_label': '11'}


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

415


## Creating embeddings
Note: Embeddings are generated on OpenAI’s servers, not locally.
### Compliance options for generating embeddings
- OpenAI API (default): Secure, encrypted, but hosted on OpenAI’s servers. Good for general use.
- Azure OpenAI Service: Same models, but hosted in Microsoft Azure with enterprise-grade compliance (GDPR, HIPAA, FedRAMP, etc.).
- Local embeddings (Hugging Face, etc.): Full control, no data leaves your environment, but requires more compute resources.

In [6]:
# !pip install -U "langchain-openai"
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [7]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 1536

[0.04834814369678497, 0.0482519268989563, 0.015562810003757477, -0.0005802979576401412, 0.026940258219838142, -0.002202426316216588, -0.009290780872106552, 0.021564234048128128, 0.003556956071406603, 0.01862967014312744]


### Store embeddings in in-memory vector store

In [8]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
ids = vector_store.add_documents(documents=all_splits)

In [9]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(len(results))
print(results[0])

4
page_content='wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,
Belgium; Taicang, China; Tomisato, Japan and Icheon, Korea, all of which we own.
Air Manufacturing Innovation manufactures cushioning components used in footwear at NIKE-owned and leased facilities located near Beaverton, Or

In [10]:
vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")

[(Document(id='56bb7546-5927-4a7a-a67f-3e9f295b1658', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './datasets/nke-10k-2023.pdf', 'total_pages': 107, 'page': 30, 'page_label': '31', 'start_index': 1938}, page_content='FINANCIAL HIGHLIGHTS\n• In fiscal 2023, NIKE, Inc. achieved record Revenues of $51.2 billion, which increased 10% and 16% on a reported and currency-neutral basis, respectively\n• NIKE Direct revenues grew 14% from $18.7 billion in fiscal 2022 to $21.3 billion in fiscal 2023, and represented approximately 44% of total NIKE Brand revenues for\nfiscal 2023\n• Gross margin for the fiscal year decreased 2

In [11]:
# Return documents based on similarity to an embedded query
embedding = embeddings.embed_query("What was Nike's revenue in 2023?")

vector_store.similarity_search_by_vector(embedding)


[Document(id='56bb7546-5927-4a7a-a67f-3e9f295b1658', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './datasets/nke-10k-2023.pdf', 'total_pages': 107, 'page': 30, 'page_label': '31', 'start_index': 1938}, page_content='FINANCIAL HIGHLIGHTS\n• In fiscal 2023, NIKE, Inc. achieved record Revenues of $51.2 billion, which increased 10% and 16% on a reported and currency-neutral basis, respectively\n• NIKE Direct revenues grew 14% from $18.7 billion in fiscal 2022 to $21.3 billion in fiscal 2023, and represented approximately 44% of total NIKE Brand revenues for\nfiscal 2023\n• Gross margin for the fiscal year decreased 25

In [14]:
## RAG agent
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [15]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a pdf. "
    "Use the tool to help answer user queries."
)
model = init_chat_model("gpt-4.1")
agent = create_agent(model, tools, system_prompt=prompt)

In [16]:
query = (
    "What was Nike's revenue in 2023?\n\n"
    "Once you get the answer, look up why the margins were impacted if any."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What was Nike's revenue in 2023?

Once you get the answer, look up why the margins were impacted if any.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_vgnJdRdrrJrac8Omt2JjrLDG)
 Call ID: call_vgnJdRdrrJrac8Omt2JjrLDG
  Args:
    query: Nike revenue 2023
================================= Tool Message =================================
Name: retrieve_context

Source: {'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './datasets/nke-10k-2023.pdf', 'total_pages': 107, 'page': 35, 'p